In [ ]:
# Los ficheros 2015.csv y 2019.csv tienen las siguientes columnas e información
# https://www.kaggle.com/datasets/unsdsn/world-happiness

#importa ambos datasets
import pandas as pd
data2015= pd.read_csv("2015.csv")
data2019= pd.read_csv("2019.csv")

# Quedate con las columnas país, región, ranking happiness, score happines.
data2015.drop(columns=[
        'Standard Error', 'Economy (GDP per Capita)', 'Family',
        'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
        'Generosity', 'Dystopia Residual'], inplace=True)
data2019.drop(columns=['GDP per capita',
        'Social support', 'Healthy life expectancy',
        'Freedom to make life choices', 'Generosity',
        'Perceptions of corruption'], inplace=True)

# Renombra a pais, region, puesto, puntuacion
data2015.columns=['Pais','Region','Puesto','Puntuacion']
data2019.columns=['Puesto','Pais','Puntuacion']

# añade una columna año con el año correspondiente
data2015['Anio'] = 2015
data2019['Anio'] = 2019

# une ambos datasets de manera que tengas las siguientes columnas
# pais, region, año, puesto, puntuacion
def region(pais):
        consulta= data2015.query(f'Pais == "{pais}"')['Region']
        try:
                return consulta.values[0]
        except:
                None

data2019['Region'] = data2019['Pais'].map(region)

data201519 = pd.concat([data2015,data2019])
print(data201519.shape)
print(data2015.shape)
print(data2019.shape)

data201519.head(5)

(314, 5)
(158, 5)
(156, 5)


,Pais,Region,Puesto,Puntuacion,Anio
0,Switzerland,Western Europe,1,7.587,2015
1,Iceland,Western Europe,2,7.561,2015
2,Denmark,Western Europe,3,7.527,2015
3,Norway,Western Europe,4,7.522,2015
4,Canada,North America,5,7.427,2015


In [ ]:
# el siguiente dataset a utilizar es el siguiente
# https://www.kaggle.com/datasets/utkarshxy/who-worldhealth-statistics-2020-complete
# utilizaremos lifeExpectancyAtBirth.csv, importa el csv
# las columnas las puedes consultar en el enlace de kaggle
data_life_expectancy= pd.read_csv('lifeExpectancyAtBirth.csv')

# Borra la columna indicator y renombra el resto a: pais, año, sexo, edad
data_life_expectancy.drop(columns=['Indicator'], inplace=True)
data_life_expectancy.columns=['Pais', 'Anio', 'Sexo', 'Edad']

# crea un dataset solamente con las filas del 2015 y 2019
filtrado= data_life_expectancy.query('Anio in (2015, 2019)').copy()
filtrado.head(5)

,Pais,Anio,Sexo,Edad
0,Afghanistan,2019,Both sexes,63.21
1,Afghanistan,2019,Male,63.29
2,Afghanistan,2019,Female,63.16
3,Afghanistan,2015,Both sexes,61.65
4,Afghanistan,2015,Male,61.04


In [18]:
#junta ambos dataframes en uno solo. Quizás sea conveniente poner todos los string a lower
finaldf= pd.merge(data201519,filtrado, how='inner', on=['Pais', 'Anio'])
#El dataframe resultante será algo así
#pais, region, año, puesto, puntuacion, sexo, edad
finaldf.head(5)

,Pais,Region,Puesto,Puntuacion,Anio,Sexo,Edad
0,Switzerland,Western Europe,1,7.587,2015,Both sexes,82.61
1,Switzerland,Western Europe,1,7.587,2015,Male,80.64
2,Switzerland,Western Europe,1,7.587,2015,Female,84.50
3,Iceland,Western Europe,2,7.561,2015,Both sexes,82.14
4,Iceland,Western Europe,2,7.561,2015,Male,80.69


In [ ]:
# Mira donde se han producido nulos. Al hacer el merge si un país tiene tilde puesta en un df
# pero en otro df no la tiene, entonces no se juntarán esas filas. 
# si hay errores habrá que descubrirlos y arreglarlos a mano



In [ ]:
# Después de arreglar errores aún puede haber nulos porque falte información
# quizás hay paises que están en 2015.csv que no estan en el resto de csv o similar.
# Elimina los nulos rellenando con valores similares a tu conveniencia. 


In [22]:
#  Mostrar los 10 primeros paises con mayor incremento de puntuación de 2015 a 2019, de forma ordenada
puntuacion2019=finaldf.loc[(finaldf['Anio'] == 2019) & (finaldf['Sexo'] == 'Both sexes'), ['Pais', 'Puntuacion']]
puntuacion2015=finaldf.loc[(finaldf['Anio'] == 2015) & (finaldf['Sexo'] == 'Both sexes'), ['Pais', 'Puntuacion']]

puntuacion2019.set_index('Pais', inplace=True)
puntuacion2015.set_index('Pais', inplace=True)

incremento = puntuacion2019 - puntuacion2015
incremento.sort_values('Puntuacion', ascending = False).head(10)

,Puntuacion
Pais,
Benin,1.543
Togo,1.246
Honduras,1.072
Burkina Faso,1.000
Hungary,0.958
Romania,0.946
Gabon,0.903
Cambodia,0.881
Guinea,0.878


In [25]:
# Mostrar el nombre del país y la espanza de vida de los 10 paises con mayor esperanza en ambos sexos
finaldf.query('Sexo == "Both sexes" and Anio == 2019')[['Pais', 'Edad']].sort_values('Edad', ascending=False).head(10)


,Pais,Edad
552,Japan,84.26
417,Switzerland,83.45
477,Spain,83.22
489,Singapore,83.22
528,Cyprus,83.14
432,Australia,83.04
495,Italy,82.97
408,Norway,82.62
438,Israel,82.62
462,France,82.48


In [26]:
# Agrupar por región y mostrar la media de la puntuación de felicidad y esperanza de vida de ambos sexos
finaldf.query('Sexo == "Both sexes"').groupby('Region')[['Puntuacion', 'Edad']].mean()

,Puntuacion,Edad
Region,,
Australia and New Zealand,7.276250,82.147500
Central and Eastern Europe,5.371458,74.884375
Eastern Asia,5.393833,76.276667
Latin America and Caribbean,6.074947,75.436579
Middle East and Northern Africa,5.531152,75.477273
North America,7.352500,82.000000
Southeastern Asia,5.365214,73.532857
Southern Asia,4.553857,70.092143
Sub-Saharan Africa,4.245462,62.869385


In [29]:
# ¿existe algún país donde la esperanza de vida sea mayor en hombres que en mujeres?
expc_mujeres=finaldf.query('Sexo == "Female" and Anio == 2019')[['Pais', 'Edad']].set_index('Pais')
expc_hombres=finaldf.query('Sexo == "Male" and Anio == 2019')[['Pais', 'Edad']].set_index('Pais')

resta= expc_hombres - expc_mujeres
resta.query('Edad >0')


,Edad
Pais,
Qatar,1.40
Afghanistan,0.13


In [33]:
# Muestra la media de la esperanza de vida de ambos sexos agrupados por región y año
finaldf.query('Sexo == "Both sexes"').groupby(['Region','Anio'])['Edad'].mean()

Region                           Anio
Australia and New Zealand        2015    81.795000
                                 2019    82.500000
Central and Eastern Europe       2015    74.383750
                                 2019    75.385000
Eastern Asia                     2015    75.956667
                                 2019    76.596667
Latin America and Caribbean      2015    74.987500
                                 2019    75.935556
Middle East and Northern Africa  2015    75.120588
                                 2019    75.856250
North America                    2015    81.760000
                                 2019    82.240000
Southeastern Asia                2015    73.261429
                                 2019    73.804286
Southern Asia                    2015    69.510000
                                 2019    70.674286
Sub-Saharan Africa               2015    62.069706
                                 2019    63.746452
Western Europe                   2015    81.

In [35]:
# Muestra la media de la esperanza de vida agrupando por año y sexo para comparar la evolución general
finaldf.groupby(['Anio', 'Sexo'])['Edad'].mean()

Anio  Sexo      
2015  Both sexes    72.339776
      Female        74.792015
      Male          69.916791
2019  Both sexes    73.216212
      Female        75.624545
      Male          70.828106
Name: Edad, dtype: float64

In [37]:
# Muestra, agrupando por país, la esperanza de vida mayor. 
finaldf.groupby('Pais')['Edad'].max().sort_values(ascending=False).head(10)

Pais
Japan          86.94
Spain          85.68
Singapore      85.45
Cyprus         85.12
France         85.09
Switzerland    85.08
Italy          84.90
Australia      84.84
Germany        84.77
Portugal       84.40
Name: Edad, dtype: float64